# Music Generation with LSTM

In [24]:
import glob
import numpy as np
import pickle
from music21 import converter, instrument, stream, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

In [25]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []
    files = "samples/*.mid"

    for file in glob.glob(files):
        midi = None
        try:
            midi = converter.parse(file)
        except:
            continue

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    pickle.dump(notes, open('data/notes.p', 'wb'))

    return notes

In [26]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 16

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [27]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    hidden_layers = 256
    dropout = 0.4
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    model.add(LayerNormalization())
    
    """ Add other layers after LSTM """
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0))

    return model

In [28]:
def train_network(load_notes=False, to_load_model=False, learning_rate=None):
    """ Train a Neural Network to generate music """
    notes = None
    model = None

    if load_notes:
        notes = pickle.load(open('data/notes.p', 'rb'))
    else:
        notes = get_notes()
    
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if to_load_model:
        model = load_model('weights.hdf5')
    else:
        model = create_network(network_input, n_vocab)

    if learning_rate != None:
        K.set_value(model.optimizer.learning_rate, learning_rate)

    checkpoint = ModelCheckpoint(
        'weights.hdf5',
        monitor='loss',
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]
    
    model.summary()

    model.fit(
        x=network_input,
        y=network_output,
        batch_size=1024,
        epochs=3000,
        callbacks=callbacks_list
    )

In [ ]:
train_network(True, True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 16, 512)           528384    
_________________________________________________________________
layer_normalization (LayerNo (None, 16, 512)           1024      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16, 512)           1574912   
_________________________________________________________________
layer_normalization_1 (Layer (None, 16, 512)           1024      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
layer_normalization_2 (Layer (None, 512)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0

Epoch 74/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4041
Epoch 75/3000
52/52 [==============================] - 6s 113ms/step - loss: 0.4032
Epoch 76/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4140
Epoch 77/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3993
Epoch 78/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3999
Epoch 79/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.4013
Epoch 80/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4015
Epoch 81/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4015
Epoch 82/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4020
Epoch 83/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4068
Epoch 84/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4047
Epoch 85/3000
52/52 [==============================] - 6s 116ms/step - loss:

52/52 [==============================] - 6s 114ms/step - loss: 0.4090
Epoch 171/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4095
Epoch 172/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3985
Epoch 173/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4089
Epoch 174/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4152
Epoch 175/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3975
Epoch 176/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4011
Epoch 177/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4001
Epoch 178/3000
52/52 [==============================] - 6s 119ms/step - loss: 0.3895
Epoch 179/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3991
Epoch 180/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4038
Epoch 181/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.

52/52 [==============================] - 6s 115ms/step - loss: 0.3971
Epoch 267/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3925
Epoch 268/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3984
Epoch 269/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3972
Epoch 270/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4039
Epoch 271/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3999
Epoch 272/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4023
Epoch 273/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3976
Epoch 274/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4004
Epoch 275/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3990
Epoch 276/3000
52/52 [==============================] - 6s 113ms/step - loss: 0.3983
Epoch 277/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.

52/52 [==============================] - 6s 114ms/step - loss: 0.3951
Epoch 363/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3959
Epoch 364/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.4041
Epoch 365/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3955
Epoch 366/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3875
Epoch 367/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3934
Epoch 368/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3925
Epoch 369/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.4023
Epoch 370/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.4003
Epoch 371/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3941
Epoch 372/3000
52/52 [==============================] - 6s 119ms/step - loss: 0.3840
Epoch 373/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.

52/52 [==============================] - 6s 115ms/step - loss: 0.3853
Epoch 459/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3870
Epoch 460/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3831
Epoch 461/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3901
Epoch 462/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3920
Epoch 463/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3857
Epoch 464/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3862
Epoch 465/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3824
Epoch 466/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3956
Epoch 467/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.3964
Epoch 468/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3879
Epoch 469/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.

52/52 [==============================] - 6s 117ms/step - loss: 0.3976
Epoch 555/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3945
Epoch 556/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3846
Epoch 557/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3898
Epoch 558/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3925
Epoch 559/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.3802
Epoch 560/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3822
Epoch 561/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3935
Epoch 562/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3938
Epoch 563/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3914
Epoch 564/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3922
Epoch 565/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.

52/52 [==============================] - 6s 114ms/step - loss: 0.3885
Epoch 651/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3826
Epoch 652/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3852
Epoch 653/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3849
Epoch 654/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3844
Epoch 655/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3839
Epoch 656/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3872
Epoch 657/3000
52/52 [==============================] - 6s 119ms/step - loss: 0.3735
Epoch 658/3000
52/52 [==============================] - 6s 118ms/step - loss: 0.3733
Epoch 659/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3806
Epoch 660/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3857
Epoch 661/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.

52/52 [==============================] - 6s 115ms/step - loss: 0.3809
Epoch 747/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.3775
Epoch 748/3000
52/52 [==============================] - 6s 115ms/step - loss: 0.3839
Epoch 749/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3852
Epoch 750/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3807
Epoch 751/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3788
Epoch 752/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3900
Epoch 753/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3882
Epoch 754/3000
52/52 [==============================] - 6s 117ms/step - loss: 0.3832
Epoch 755/3000
52/52 [==============================] - 6s 114ms/step - loss: 0.3751
Epoch 756/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.3791
Epoch 757/3000
52/52 [==============================] - 6s 116ms/step - loss: 0.

In [30]:
def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 16
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [35]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.random.choice(prediction.shape[1], p=(prediction[0] / sum(prediction[0])))
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [36]:
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            new_rest = note.Rest(pattern)
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [37]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    notes = pickle.load(open('data/notes.p', 'rb'))

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
    model = load_model('weights.hdf5')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [38]:
generate()